In [5]:
from IPython.display import HTML

def hover(hover_color="#ffff99"):
    return dict(selector="tr:hover",
                props=[("background-color", "%s" % hover_color)])

styles = [
    hover(),
    dict(selector="th", props=[("font-size", "100%"),
                               ("text-align", "center")]),
    dict(selector="caption", props=[("caption-side", "bottom")])
]
# html1 = (valve.style.set_table_styles(styles))
# html2 = (rctTemp.style.set_table_styles(styles))


In [6]:
import sys
import time
from IPython.display import display,HTML,clear_output
def dashboard(run='004', updateSec=3):

    cursor = connectDbase('rct007')

    tbName0 = 'rct007_testID'
    tbName1 = run+'_flowindata'
    tbName2 = run+'_flowoutdata'
    tbName3 = run+'_plcdata'
    tbName4 = run+'_voltagedata'

    while(True):
        time.sleep(updateSec)

        cursor = connectDbase('rct007')
        plc   = readLastNTable(cursor, tbName3,1)
        volt  = readLastNTable(cursor, tbName4,1)
        flin  = readLastNTable(cursor, tbName1)
        flout = readLastNTable(cursor, tbName2)

        valve = plc.loc[:,['maingasOpen', 'SysRctOpen', 'EvacOpen', 'HivacOpen', 'ForelineOpen', 'RoughOpen', 'sysRgaOpen']]
        valve.rename(columns={'maingasOpen': 'MainGas', 'SysRctOpen': 'Sys RCT', 'EvacOpen': 'Gas Evac',
                              'HivacOpen': 'Hivac', 'ForelineOpen': 'Foreline', 'RoughOpen': 'Rough'}, inplace=True)
        for column in valve:
            if np.any(valve[column] == 1):
                valve[column] = 'Open'
            else:
                valve[column] = 'Closed'
        vhtml = (valve.style.set_table_styles(styles)).set_caption('Valve State')
        

        rctTemp = plc.loc[:, ['rct_rtd1', 'rct_rtd2', 'rct_rtd3', 'rct_rtd4', 'rct_rtd5', 'rct_rtd6']]
        rctThtml = (rctTemp.style.set_table_styles(styles)).set_caption('Reactor Temperatures, degrees C')

        ambTemp = plc.loc[:, ['amb_rtd1', 'amb_rtd2', 'amb_rtd3', 'amb_rtd4']]
        ambThtml = (ambTemp.style.set_table_styles(styles)).set_caption('Ambient Temperatures, degrees C')

        press = plc.loc[:, ['BaratronPress', 'forelinePress', 'systemPress', '20TBaratronPress']]
        press.rename(columns={'BaratronPress': 'Baratron Pressure (Torr)', 'forelinePress': 
                            'Foreline Pressure (Torr)', 'systemPress':'System Pressure (Torr)', 
                            '20TBaratronPress': '20T Baratron Pressure (Torr)'}, inplace = True)
        press = press.reindex(index=press.index[::-1])                    
        phtml = (press.style.set_table_styles(styles)).set_caption('Pressure Readings')

        gasFlow = plc.loc[:, ['H2Flowrate', 'D2Flowrate', 'ARFlowrate', 'HeFlowrate']]
        ghtml = (gasFlow.style.set_table_styles(styles).set_caption('Gas Flow Rate, SCCM'))
        
        heat = plc.loc[:, ['htrstate', 'htrtimeon', 'htrperiod', 'htr1duty', 'htr2duty','htr1_tc','htr2_tc']]
        heat.rename(columns={'htrperiod':'Period', 'htrstate':'State', 'htr1duty':'Duty Cycle H1',
                             'htr2duty':'Duty Cycle H2','htrtimeon':'Time On','htr1_tc':'Temp H1',
                             'htr2_tc':'Temp H2'}, inplace=True)                    
        hhtml = (heat.style.set_table_styles(styles)).set_caption('Heater Settings')

        flow = plc.loc[:,['flowin_rtd','flowout_rtd'] ]
        flow['rate in'] = flin.flowin[0]
        flow['rate out'] = flout.flowout[0]
        fhtml = (flow.style.set_table_styles(styles).set_caption('Water Cooler Flow'))

        clear_output(wait=True)

        display(HTML('<font size=4><b>Kernel -> Interrupt To Stop</b></font>'))
        display((vhtml))
        display((phtml))
        display((ghtml))
        display((rctThtml))
        display((ambThtml))
        display((hhtml))
        display((fhtml))
        